In [1]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
import transformers
import un_ner_tokens as get_tokens
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np

import torch
print(torch.cuda.is_available())

label_list = [
    'O',       # Outside of a named entity
    'B-MISC',  # Beginning of a miscellaneous entity right after another miscellaneous entity
    'I-MISC',  # Miscellaneous entity
    'B-PER',   # Beginning of a person's name right after another person's name
    'I-PER',   # Person's name
    'B-ORG',   # Beginning of an organisation right after another organisation
    'I-ORG',   # Organisation
    'B-LOC',   # Beginning of a location right after another location
    'I-LOC'    # Location
]

tokenizer = AutoTokenizer.from_pretrained('./un-ner.model/')

False


In [2]:
tokenizer

PreTrainedTokenizerFast(name_or_path='./un-ner.model/', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [3]:
tokens = tokenizer('''Members will recall that, at its 2nd plenary meeting, on 20 September 2019, the assembly decided to include this item in the agenda of the seventy-fourth session. In connection with this item, I have received a letter dated 27 august 2020 from the deputy permanent representative of Brazil to the United Nations requesting that the item be included in the draft agenda of the seventy-fifth session of the assembly. I give the floor to the representative of Armenia. Members will recall that at its 2nd plenary meeting, on 20 September 2019, the assembly decided to include this item in the agenda of the seventy-fourth session. In connection with the item, a letter dated 31 august 2020 from the permanent representative of the Russian federation to the United Nations addressed to the president of the assembly has been issued as document a/74/1002, in which it is requested that the item be included in the agenda of the seventy-fifth session of the assembly. Members will recall that at its 2nd plenary meeting, on 20 September 2019, the assembly decided to include this item in the agenda of the seventy - fourth session. My delegation would like to disassociate itself from the decision to include agenda item 37 on the draft agenda of the seventy-fifth session of the general assembly. The assembly has before it five draft resolutions recommended by the third committee in paragraph 47 of its report. Before proceeding further, I should like to inform members that action on draft resolution iv, entitled situation of human rights of Rohingya Muslims and other minorities in Myanmar is postponed to a later date to allow time for the review of its programme budget implications by the fifth committee. The assembly will take action on draft resolution iv as soon as the report of the fifth committee on the programme budget implications is available. I now give the floor to delegations wishing to deliver explanations of vote or position before voting or adoption.''')
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

torch.Size([1, 379])

In [4]:
model = AutoModelForTokenClassification.from_pretrained('./un-ner.model/', num_labels=len(label_list))

In [5]:
import torch
preds = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))

In [6]:
preds = torch.argmax(preds.logits.squeeze(), axis=1)

In [7]:
sentence = '''Members will recall that, at its 2nd plenary meeting, on 20 September 2019, the assembly decided to include this item in the agenda of the seventy-fourth session. In connection with this item, I have received a letter dated 27 august 2020 from the deputy permanent representative of Brazil to the United Nations requesting that the item be included in the draft agenda of the seventy-fifth session of the assembly. I give the floor to the representative of Armenia. Members will recall that at its 2nd plenary meeting, on 20 September 2019, the assembly decided to include this item in the agenda of the seventy-fourth session. In connection with the item, a letter dated 31 august 2020 from the permanent representative of the Russian federation to the United Nations addressed to the president of the assembly has been issued as document a/74/1002, in which it is requested that the item be included in the agenda of the seventy-fifth session of the assembly. Members will recall that at its 2nd plenary meeting, on 20 September 2019, the assembly decided to include this item in the agenda of the seventy - fourth session. My delegation would like to disassociate itself from the decision to include agenda item 37 on the draft agenda of the seventy-fifth session of the general assembly. The assembly has before it five draft resolutions recommended by the third committee in paragraph 47 of its report. Before proceeding further, I should like to inform members that action on draft resolution iv, entitled situation of human rights of Rohingya Muslims and other minorities in Myanmar is postponed to a later date to allow time for the review of its programme budget implications by the fifth committee. The assembly will take action on draft resolution iv as soon as the report of the fifth committee on the programme budget implications is available. I now give the floor to delegations wishing to deliver explanations of vote or position before voting or adoption.'''

In [8]:
len(sentence.split(' '))

329

In [9]:
preds.size()

torch.Size([379])

In [10]:
len(tokens['input_ids'])

379

In [11]:
words = tokenizer.batch_decode(tokens['input_ids'])

In [12]:
value_preds = [label_list[i] for i in preds]

In [13]:
import pandas as pd
pd.DataFrame({'ner': value_preds, 'words': words}).to_csv('un_ner.csv')